In [1]:
import pandas as pd
from pyhive import hive
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import threading
import time


2023-12-02 22:22:59.722531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def read_hive_data(thread_id):
    global train_data, test_data
    # Hive loading data
    hive_host = "master"
    hive_port = 10000 
    hive_username = "root" 
    hive_password = "root" 
    hive_database = "default"  

    # Creating Hive connection
    conn = hive.Connection(
        host=hive_host,
        port=hive_port,
        username=hive_username,
        password=hive_password,
        database=hive_database,
        auth='CUSTOM',
    )
    
    cursor = conn.cursor()

    query_train = "SELECT * FROM bike_train"
    cursor.execute(query_train)

    result_train = cursor.fetchall()
    train_data = pd.DataFrame(result_train, columns=[desc[0] for desc in cursor.description])

    query_test = "SELECT * FROM bike_test"
    cursor.execute(query_test)
    result_test = cursor.fetchall()
    test_data = pd.DataFrame(result_test, columns=[desc[0] for desc in cursor.description])


# parallel_read_hive_data
def parallel_read_hive_data(num_threads):
    threads = []

    for i in range(num_threads):
        thread = threading.Thread(target=read_hive_data, args=(i,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()


start_parallel = time.time()
parallel_read_hive_data(num_threads=8)
end_parallel = time.time()
print("Time cost with parallel method：{:.2f}s".format(end_parallel - start_parallel))

Time cost with parallel method：0.84s


In [8]:
#Without parallel_read_hive_data
start = time.time()
hive_host = "master"
hive_port = 10000 
hive_username = "root" 
hive_password = "root" 
hive_database = "default" 

conn = hive.Connection(
    host=hive_host,
    port=hive_port,
    username=hive_username,
    password=hive_password,
    database=hive_database,
    auth='CUSTOM',
)

cursor = conn.cursor()

query_train = "SELECT * FROM bike_train"
cursor.execute(query_train)

result_train = cursor.fetchall()
train_data = pd.DataFrame(result_train, columns=[desc[0] for desc in cursor.description])

query_test = "SELECT * FROM bike_test"
cursor.execute(query_test)
result_test = cursor.fetchall()

test_data = pd.DataFrame(result_test, columns=[desc[0] for desc in cursor.description])
end = time.time()
print("Time cost without parallel method：{:.2f}s".format(end - start))

Time cost without parallel method：2.27s


In [29]:
train_data.columns = train_data.columns.str.replace('bike_train.', '')
test_data.columns = test_data.columns.str.replace('bike_test.', '')

In [58]:
X_train = train_data.drop(['date', 'casual', 'registered', 'count'], axis=1).values
y_train = train_data['count'].values
X_test = test_data.drop(['date', 'casual', 'registered', 'count'], axis=1).values

In [ ]:
from keras import backend as K
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.01), metrics=[coeff_determination])

model.fit(X_train, y_train, batch_size=128, epochs=5000, verbose=1)

predictions = model.predict(X_test)

Epoch 1/5000
86/86 [==============================] - 1s 2ms/step - loss: 29110.3262 - coeff_determination: 0.1066
Epoch 2/5000
86/86 [==============================] - 0s 2ms/step - loss: 21997.9297 - coeff_determination: 0.3165
Epoch 3/5000
86/86 [==============================] - 0s 2ms/step - loss: 21839.2246 - coeff_determination: 0.0441
Epoch 4/5000
86/86 [==============================] - 0s 2ms/step - loss: 21673.4785 - coeff_determination: 0.3331
Epoch 5/5000
86/86 [==============================] - 0s 2ms/step - loss: 21077.1309 - coeff_determination: 0.3479
Epoch 6/5000
86/86 [==============================] - 0s 2ms/step - loss: 21069.8359 - coeff_determination: 0.3463
Epoch 7/5000
86/86 [==============================] - 0s 2ms/step - loss: 21644.8594 - coeff_determination: 0.3329
Epoch 8/5000
86/86 [==============================] - 0s 2ms/step - loss: 20651.0879 - coeff_determination: 0.3634
Epoch 9/5000
86/86 [==============================] - 0s 2ms/step - loss: 20487.

86/86 [==============================] - 0s 2ms/step - loss: 17595.1152 - coeff_determination: 0.4556
Epoch 72/5000
86/86 [==============================] - 0s 1ms/step - loss: 17596.3945 - coeff_determination: 0.4588
Epoch 73/5000
86/86 [==============================] - 0s 1ms/step - loss: 17456.7227 - coeff_determination: 0.4632
Epoch 74/5000
86/86 [==============================] - 0s 1ms/step - loss: 17492.2246 - coeff_determination: 0.4617
Epoch 75/5000
86/86 [==============================] - 0s 1ms/step - loss: 17573.8145 - coeff_determination: 0.4629
Epoch 76/5000
86/86 [==============================] - 0s 2ms/step - loss: 17457.6133 - coeff_determination: 0.4647
Epoch 77/5000
86/86 [==============================] - 0s 1ms/step - loss: 17900.9688 - coeff_determination: 0.3980
Epoch 78/5000
86/86 [==============================] - 0s 1ms/step - loss: 17494.0000 - coeff_determination: 0.4552
Epoch 79/5000
86/86 [==============================] - 0s 1ms/step - loss: 17564.5020 

86/86 [==============================] - 0s 2ms/step - loss: 15191.0693 - coeff_determination: 0.5289
Epoch 142/5000
86/86 [==============================] - 0s 2ms/step - loss: 15118.9199 - coeff_determination: 0.5294
Epoch 143/5000
86/86 [==============================] - 0s 2ms/step - loss: 15170.7852 - coeff_determination: 0.5270
Epoch 144/5000
86/86 [==============================] - 0s 2ms/step - loss: 15205.0088 - coeff_determination: 0.5339
Epoch 145/5000
86/86 [==============================] - 0s 2ms/step - loss: 15127.0430 - coeff_determination: 0.5252
Epoch 146/5000
86/86 [==============================] - 0s 2ms/step - loss: 15144.7617 - coeff_determination: 0.5315
Epoch 147/5000
86/86 [==============================] - 0s 2ms/step - loss: 15501.5068 - coeff_determination: 0.4984
Epoch 148/5000
86/86 [==============================] - 0s 2ms/step - loss: 15143.4238 - coeff_determination: 0.5273
Epoch 149/5000
86/86 [==============================] - 0s 3ms/step - loss: 153

86/86 [==============================] - 0s 1ms/step - loss: 13359.4150 - coeff_determination: 0.5377
Epoch 212/5000
86/86 [==============================] - 0s 1ms/step - loss: 13574.9648 - coeff_determination: 0.5778
Epoch 213/5000
86/86 [==============================] - 0s 1ms/step - loss: 13742.6025 - coeff_determination: 0.5662
Epoch 214/5000
86/86 [==============================] - 0s 1ms/step - loss: 13607.0811 - coeff_determination: 0.5745
Epoch 215/5000
86/86 [==============================] - 0s 1ms/step - loss: 13389.4297 - coeff_determination: 0.5757
Epoch 216/5000
86/86 [==============================] - 0s 1ms/step - loss: 13628.4902 - coeff_determination: 0.5810
Epoch 217/5000
86/86 [==============================] - 0s 1ms/step - loss: 13735.0244 - coeff_determination: 0.5705
Epoch 218/5000
86/86 [==============================] - 0s 1ms/step - loss: 13445.4912 - coeff_determination: 0.5753
Epoch 219/5000
86/86 [==============================] - 0s 1ms/step - loss: 134

86/86 [==============================] - 0s 1ms/step - loss: 12498.1035 - coeff_determination: 0.6087
Epoch 282/5000
86/86 [==============================] - 0s 1ms/step - loss: 12257.6406 - coeff_determination: 0.6166
Epoch 283/5000
86/86 [==============================] - 0s 2ms/step - loss: 12650.9971 - coeff_determination: 0.5813
Epoch 284/5000
86/86 [==============================] - 0s 1ms/step - loss: 12689.8516 - coeff_determination: 0.6100
Epoch 285/5000
86/86 [==============================] - 0s 1ms/step - loss: 12142.2881 - coeff_determination: 0.6166
Epoch 286/5000
86/86 [==============================] - 0s 1ms/step - loss: 13002.6826 - coeff_determination: 0.5996
Epoch 287/5000
86/86 [==============================] - 0s 1ms/step - loss: 12439.5195 - coeff_determination: 0.6125
Epoch 288/5000
86/86 [==============================] - 0s 1ms/step - loss: 12239.5117 - coeff_determination: 0.6166
Epoch 289/5000
86/86 [==============================] - 0s 1ms/step - loss: 124

...

86/86 [==============================] - 0s 2ms/step - loss: 6294.8320 - coeff_determination: 0.8041
Epoch 4692/5000
86/86 [==============================] - 0s 2ms/step - loss: 6186.0269 - coeff_determination: 0.8091
Epoch 4693/5000
86/86 [==============================] - 0s 2ms/step - loss: 6332.4995 - coeff_determination: 0.8009
Epoch 4694/5000
86/86 [==============================] - 0s 1ms/step - loss: 6358.9570 - coeff_determination: 0.8015
Epoch 4695/5000
86/86 [==============================] - 0s 1ms/step - loss: 6123.8730 - coeff_determination: 0.8070
Epoch 4696/5000
86/86 [==============================] - 0s 1ms/step - loss: 6433.4961 - coeff_determination: 0.8020
Epoch 4697/5000
86/86 [==============================] - 0s 2ms/step - loss: 6193.7290 - coeff_determination: 0.8055
Epoch 4698/5000
86/86 [==============================] - 0s 1ms/step - loss: 6331.3057 - coeff_determination: 0.8018
Epoch 4699/5000
86/86 [==============================] - 0s 1ms/step - loss: 634

86/86 [==============================] - 0s 1ms/step - loss: 6315.1445 - coeff_determination: 0.8043
Epoch 4762/5000
86/86 [==============================] - 0s 2ms/step - loss: 6160.3926 - coeff_determination: 0.8095
Epoch 4763/5000
86/86 [==============================] - 0s 2ms/step - loss: 6177.5029 - coeff_determination: 0.8071
Epoch 4764/5000
86/86 [==============================] - 0s 1ms/step - loss: 5966.4492 - coeff_determination: 0.8126
Epoch 4765/5000
86/86 [==============================] - 0s 1ms/step - loss: 7013.6265 - coeff_determination: 0.7840
Epoch 4766/5000
86/86 [==============================] - 0s 1ms/step - loss: 6478.0444 - coeff_determination: 0.7991
Epoch 4767/5000
86/86 [==============================] - 0s 1ms/step - loss: 5946.7725 - coeff_determination: 0.8165
Epoch 4768/5000
86/86 [==============================] - 0s 1ms/step - loss: 5996.6001 - coeff_determination: 0.8052
Epoch 4769/5000
86/86 [==============================] - 0s 1ms/step - loss: 636

86/86 [==============================] - 0s 1ms/step - loss: 7694.0107 - coeff_determination: 0.7627
Epoch 4832/5000
86/86 [==============================] - 0s 1ms/step - loss: 6852.3828 - coeff_determination: 0.7864
Epoch 4833/5000
86/86 [==============================] - 0s 1ms/step - loss: 7075.4614 - coeff_determination: 0.7791
Epoch 4834/5000
86/86 [==============================] - 0s 1ms/step - loss: 6500.0347 - coeff_determination: 0.7979
Epoch 4835/5000
86/86 [==============================] - 0s 1ms/step - loss: 6850.0518 - coeff_determination: 0.7866
Epoch 4836/5000
86/86 [==============================] - 0s 1ms/step - loss: 7231.3945 - coeff_determination: 0.7752
Epoch 4837/5000
86/86 [==============================] - 0s 1ms/step - loss: 6301.3428 - coeff_determination: 0.7953
Epoch 4838/5000
86/86 [==============================] - 0s 1ms/step - loss: 6211.1650 - coeff_determination: 0.8027
Epoch 4839/5000
86/86 [==============================] - 0s 1ms/step - loss: 688

86/86 [==============================] - 0s 1ms/step - loss: 6108.1953 - coeff_determination: 0.8084
Epoch 4902/5000
86/86 [==============================] - 0s 1ms/step - loss: 6383.3638 - coeff_determination: 0.7993
Epoch 4903/5000
86/86 [==============================] - 0s 1ms/step - loss: 6452.5161 - coeff_determination: 0.7971
Epoch 4904/5000
86/86 [==============================] - 0s 1ms/step - loss: 6342.9497 - coeff_determination: 0.8033
Epoch 4905/5000
86/86 [==============================] - 0s 1ms/step - loss: 6117.5806 - coeff_determination: 0.8097
Epoch 4906/5000
86/86 [==============================] - 0s 1ms/step - loss: 6184.5410 - coeff_determination: 0.8073
Epoch 4907/5000
86/86 [==============================] - 0s 1ms/step - loss: 5929.1123 - coeff_determination: 0.8156
Epoch 4908/5000
86/86 [==============================] - 0s 1ms/step - loss: 6379.8403 - coeff_determination: 0.8023
Epoch 4909/5000
86/86 [==============================] - 0s 2ms/step - loss: 609

86/86 [==============================] - 0s 1ms/step - loss: 6454.9644 - coeff_determination: 0.7967
Epoch 4972/5000
86/86 [==============================] - 0s 1ms/step - loss: 6154.8384 - coeff_determination: 0.8090
Epoch 4973/5000
86/86 [==============================] - 0s 1ms/step - loss: 6031.4248 - coeff_determination: 0.8109
Epoch 4974/5000
86/86 [==============================] - 0s 1ms/step - loss: 6573.0649 - coeff_determination: 0.7969
Epoch 4975/5000
86/86 [==============================] - 0s 1ms/step - loss: 5969.3765 - coeff_determination: 0.8144
Epoch 4976/5000
86/86 [==============================] - 0s 1ms/step - loss: 6129.4316 - coeff_determination: 0.8035
Epoch 4977/5000
86/86 [==============================] - 0s 1ms/step - loss: 6383.9316 - coeff_determination: 0.8024
Epoch 4978/5000
86/86 [==============================] - 0s 1ms/step - loss: 6104.6924 - coeff_determination: 0.8024
Epoch 4979/5000
86/86 [==============================] - 0s 1ms/step - loss: 655

In [62]:
result = (pd.DataFrame({'date': test_data['date'],
                        'count': test_data['count'],
                        'prediction': predictions.flatten()})
          .groupby('date').sum())
result.head()

,count,prediction
date,,
2011-01-20,0,1299.282471
2011-01-21,0,1531.058350
2011-01-22,0,870.610474
2011-01-23,0,890.229614
2011-01-24,0,1453.618530
